In [2]:
from utilities import get_key

from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker


client = QdrantClient(
    url=get_key("QDRANT_URL"),
    api_key=get_key("QDRANT_KEY"),
)
client

In [4]:
dummy_data_collection = "dummy_collection"

In [4]:
# Fake User Data
dummy_data_genrator = Faker()
dummy_data_genrator.name(), dummy_data_genrator.address(), dummy_data_genrator.country(), dummy_data_genrator.color()


('Travis Baker', 'PSC 1212, Box 4212\nAPO AE 57116', 'Albania', '#d35ae2')

In [5]:
vector_data_size = 100
vector_data = np.random.uniform(low=-1.0,high=1.0, size=(vector_data_size,100))
index = list(range(vector_data_size))

In [6]:
payload_data = []

for r in range(vector_data_size):
    payload_data.append(
        {
            "name":dummy_data_genrator.name(),
            "address":dummy_data_genrator.address(),
            "country":dummy_data_genrator.country(),
            "url":dummy_data_genrator.url(),
            "year":dummy_data_genrator.year(),
            "color":dummy_data_genrator.color()
        }
    )
    
payload_data[:2]

[{'name': 'Candice Garcia',
  'address': '533 Booth Mountains Apt. 983\nPeterbury, NC 66735',
  'country': 'Canada',
  'url': 'http://miller.info/',
  'year': '1987',
  'color': '#9fe544'},
 {'name': 'Joel Lee',
  'address': '71518 Jasmine Cape Suite 334\nWest Sarah, TX 17659',
  'country': 'New Zealand',
  'url': 'http://www.castillo-martinez.biz/',
  'year': '1980',
  'color': '#ed1225'}]

In [12]:
try:
    client.create_collection(
        collection_name=dummy_data_collection,
        vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
    )
except Exception as ex:
    print(ex)

Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `dummy_collection` already exists!"},"time":0.009697678}'


In [8]:
client.upsert(
    collection_name=dummy_data_collection,
    points=models.Batch(
        ids=index,
        vectors=vector_data.tolist(),
        payloads=payload_data
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
serch_input_vector = np.random.uniform(low=-1.0,high=1.0, size=(100))

client.search(
    collection_name=dummy_data_collection,
    query_vector=serch_input_vector, # Random vector
    limit=15
)

[ScoredPoint(id=79, version=0, score=0.24612512, payload={'address': 'Unit 4654 Box 3598\nDPO AP 71186', 'color': '#23fcb7', 'country': 'Senegal', 'name': 'Sylvia Robertson', 'url': 'http://lindsey-simpson.com/', 'year': '1989'}, vector=None, shard_key=None),
 ScoredPoint(id=73, version=0, score=0.21954285, payload={'address': '1401 Harris Place\nJohnburgh, NC 52917', 'color': '#67e5a4', 'country': 'Jamaica', 'name': 'Diane Walter', 'url': 'https://www.bailey-soto.biz/', 'year': '1981'}, vector=None, shard_key=None),
 ScoredPoint(id=90, version=0, score=0.2154387, payload={'address': '149 Brennan Ports\nWest Robertstad, WV 97451', 'color': '#e8d153', 'country': 'Nauru', 'name': 'Daniel Robinson', 'url': 'https://www.lambert-barnett.com/', 'year': '1982'}, vector=None, shard_key=None),
 ScoredPoint(id=67, version=0, score=0.20277382, payload={'address': '8649 Terry Flats\nMendozamouth, WI 05794', 'color': '#3cc19e', 'country': 'Syrian Arab Republic', 'name': 'Brian White', 'url': 'https

In [13]:

client.search(
    collection_name=dummy_data_collection,
    query_vector=vector_data[0], # 100% matching vector
    limit=15
)

[ScoredPoint(id=0, version=0, score=1.0, payload={'address': '533 Booth Mountains Apt. 983\nPeterbury, NC 66735', 'color': '#9fe544', 'country': 'Canada', 'name': 'Candice Garcia', 'url': 'http://miller.info/', 'year': '1987'}, vector=None, shard_key=None),
 ScoredPoint(id=95, version=0, score=0.1931763, payload={'address': 'PSC 0849, Box 0570\nAPO AE 15418', 'color': '#2baf51', 'country': 'Italy', 'name': 'William Gutierrez', 'url': 'http://martin.org/', 'year': '2001'}, vector=None, shard_key=None),
 ScoredPoint(id=9, version=0, score=0.18886496, payload={'address': '45937 Perez Divide\nAndersonfort, WA 34103', 'color': '#78a1e8', 'country': 'Bosnia and Herzegovina', 'name': 'Hector Singleton', 'url': 'http://reed.com/', 'year': '1988'}, vector=None, shard_key=None),
 ScoredPoint(id=89, version=0, score=0.17631654, payload={'address': 'Unit 3418 Box 6264\nDPO AE 83303', 'color': '#93eaf9', 'country': 'Kazakhstan', 'name': 'Mrs. Emily Phillips DDS', 'url': 'https://www.welch.net/', 'y

In [18]:
# 100% baised result
payload_data[0], payload_data[0]["year"], vector_data[0][:4]

({'name': 'Candice Garcia',
  'address': '533 Booth Mountains Apt. 983\nPeterbury, NC 66735',
  'country': 'Canada',
  'url': 'http://miller.info/',
  'year': '1987',
  'color': '#9fe544'},
 '1987',
 array([ 0.19829526, -0.65049326, -0.85415402,  0.05506963]))

In [20]:
# Custom filters
custom_filter_100_percent = models.Filter(
    must=[
        models.FieldCondition(
            key="year", match=models.MatchValue(value=payload_data[0]["year"])
        )
    ]
)

custom_filter_random = models.Filter(
    must=[
        models.FieldCondition(
            key="year", match=models.MatchValue(value="1980")
        )
    ]
)

custom_filter_100_percent, custom_filter_random

(Filter(should=None, min_should=None, must=[FieldCondition(key='year', match=MatchValue(value='1987'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None),
 Filter(should=None, min_should=None, must=[FieldCondition(key='year', match=MatchValue(value='1980'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None))

In [21]:
# Search with custom filter baised
client.search(
    collection_name=dummy_data_collection,
    query_vector=vector_data[0], # 100% matching vector,
    query_filter= custom_filter_100_percent,
    limit=15
)

[ScoredPoint(id=0, version=0, score=1.0, payload={'address': '533 Booth Mountains Apt. 983\nPeterbury, NC 66735', 'color': '#9fe544', 'country': 'Canada', 'name': 'Candice Garcia', 'url': 'http://miller.info/', 'year': '1987'}, vector=None, shard_key=None)]

In [22]:
client.search(
    collection_name=dummy_data_collection,
    query_vector=serch_input_vector, # Random vector
    query_filter= custom_filter_random,
    limit=15
)

[ScoredPoint(id=21, version=0, score=0.042913154, payload={'address': '6135 William Shoal Apt. 595\nEast Jamesfort, IN 58664', 'color': '#8dd33d', 'country': 'Christmas Island', 'name': 'Suzanne Richards', 'url': 'http://browning-archer.com/', 'year': '1980'}, vector=None, shard_key=None),
 ScoredPoint(id=74, version=0, score=-0.11570884, payload={'address': '75215 Dickerson Harbors\nPort Lisa, SD 79868', 'color': '#5c167f', 'country': 'Portugal', 'name': 'Veronica Morris', 'url': 'https://richardson-rios.com/', 'year': '1980'}, vector=None, shard_key=None),
 ScoredPoint(id=1, version=0, score=-0.1800782, payload={'address': '71518 Jasmine Cape Suite 334\nWest Sarah, TX 17659', 'color': '#ed1225', 'country': 'New Zealand', 'name': 'Joel Lee', 'url': 'http://www.castillo-martinez.biz/', 'year': '1980'}, vector=None, shard_key=None)]

In [29]:
# Recommendation Engine

client.recommend(
    collection_name=dummy_data_collection,
    positive=[1], # index of similar pattern
    negative=[75,40], # negitive index
    query_filter= custom_filter_random, # custom filter on day they were born
    limit=5
)


[ScoredPoint(id=21, version=0, score=0.067769356, payload={'address': '6135 William Shoal Apt. 595\nEast Jamesfort, IN 58664', 'color': '#8dd33d', 'country': 'Christmas Island', 'name': 'Suzanne Richards', 'url': 'http://browning-archer.com/', 'year': '1980'}, vector=None, shard_key=None),
 ScoredPoint(id=74, version=0, score=-0.03420703, payload={'address': '75215 Dickerson Harbors\nPort Lisa, SD 79868', 'color': '#5c167f', 'country': 'Portugal', 'name': 'Veronica Morris', 'url': 'https://richardson-rios.com/', 'year': '1980'}, vector=None, shard_key=None)]